In [2]:
import numpy as np
import sympy as sym
import pydae.build as db

In [15]:
params_dict = {'L':5.21,'G':9.81,'M':10.0,'K_d':1e-6}


u_ini_dict = {'theta':np.deg2rad(5.0)}  # for the initialization problem
u_run_dict = {'f_x':10}  # for the running problem (here initialization and running problem are the same)

#u_ini_dict = {'theta':10}  # for the initialization problem
u_run_dict = {'f_x':10}  # for the running problem (here initialization and running problem are the same)



x_list = ['pos_x','pos_y','v_x','v_y']    # [inductor current, PI integrator]
y_ini_list = ['lam','f_x'] # for the initialization problem
y_run_list = ['lam','theta'] # for the running problem (here initialization and running problem are the same)

sys_vars = {'params':params_dict,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [16]:
dpos_x = v_x
dpos_y = v_y
dv_x = (-2*pos_x*lam + f_x - K_d*v_x)/M
dv_y = (-M*G - 2*pos_y*lam - K_d*v_y)/M   

g_1 = pos_x**2 + pos_y**2 - L**2 -lam*1e-6
g_2 = -theta + sym.atan2(pos_x,-pos_y)

In [17]:
sys = {'name':'pendulum',
       'params_dict':params_dict,
       'f_list':[dpos_x,dpos_y,dv_x,dv_y],
       'g_list':[g_1,g_2],
       'x_list':x_list,
       'y_ini_list':y_ini_list,
       'y_run_list':y_run_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':{'g_1':g_1,'PE':M*G*pos_y,'KE':0.5*M*(v_x**2+v_y**2),'theta':theta}}

sys = db.system(sys)
db.sys2num(sys)

In [ ]:
g = -I + Y @ V
g_p = -p + i*v 
g_q = -q + i*v 
y = V

G_y = jacobiano(g,y)